In [28]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import folium

In [29]:
import pandas as pd

pd.set_option('display.max_rows', None)  # show all rows
pd.set_option('display.max_columns', None)  # show all columns

df = pd.read_csv("data/train_master.csv")
df  # now all rows will be displayed


,barangay,population,land_area_km2,latitude,longitude,jun_per_capita,jun_sw,jun_density,jun_is_hotspot,jul_per_capita,jul_sw,jul_density,jul_is_hotspot,aug_per_capita,aug_sw,aug_density,aug_is_hotspot,sep_per_capita,sep_sw,sep_density,sep_is_hotspot,oct_per_capita,oct_sw,oct_density,oct_is_hotspot,nov_per_capita,nov_sw,nov_density,nov_is_hotspot,dec_density
0,Agusan,18485,6.28,8.48524,124.74900,0.42,7807.52,1243.63,NaN,0.44,8183.37,1303.50,NaN,0.46,8553.07,1362.39,NaN,0.41,7497.85,1194.31,NaN,0.41,7515.04,1197.04,NaN,0.44,8179.22,1302.84,NaN,NaN
1,Baikingon,2930,5.57,8.46568,124.57111,0.47,1390.70,249.54,NaN,0.46,1361.40,244.28,NaN,0.45,1326.24,237.98,NaN,0.47,1382.44,248.06,NaN,0.47,1384.33,248.40,NaN,0.48,1401.90,251.55,NaN,NaN
2,Balubal,6700,15.94,8.45186,124.77715,0.48,3208.67,201.30,NaN,0.46,3061.27,192.05,NaN,0.44,2980.87,187.01,NaN,0.48,3211.58,201.48,NaN,0.48,3202.90,200.93,NaN,0.48,3215.16,201.70,NaN,NaN
3,Balulang,43315,7.19,8.44131,124.62700,0.41,17548.71,2441.39,NaN,0.42,18328.38,2549.86,NaN,0.45,19627.83,2730.64,NaN,0.41,17563.41,2443.43,NaN,0.43,18436.95,2564.96,NaN,0.44,18862.77,2624.20,NaN,NaN
4,Barangay 1,180,0.09,8.47524,124.64269,0.46,82.99,958.58,NaN,0.44,79.75,921.16,NaN,0.45,81.37,939.87,NaN,0.46,82.99,958.58,NaN,0.47,84.92,980.84,NaN,0.48,86.17,995.31,NaN,NaN
5,Barangay 2,60,0.04,8.47629,124.64251,0.48,28.75,800.17,NaN,0.45,27.13,755.08,NaN,0.43,25.93,721.68,NaN,0.48,28.75,800.17,NaN,0.48,28.86,803.28,NaN,0.49,29.13,810.69,NaN,NaN
6,Barangay 3,110,2542.00,8.47616,124.64526,0.44,48.69,0.02,NaN,0.46,50.89,0.02,NaN,0.47,51.99,0.02,NaN,0.45,49.79,0.02,NaN,0.46,50.85,0.02,NaN,0.47,51.42,0.02,NaN,NaN
7,Barangay 4,80,1296.00,8.47674,124.64341,0.47,37.43,0.03,NaN,0.45,36.31,0.03,NaN,0.47,37.27,0.03,NaN,0.42,33.43,0.03,NaN,0.42,33.45,0.03,NaN,0.43,34.13,0.03,NaN,NaN
8,Barangay 5,50,984.80,8.47745,124.64374,0.42,21.09,0.02,NaN,0.40,20.09,0.02,NaN,0.42,21.09,0.02,NaN,0.41,20.59,0.02,NaN,0.41,20.61,0.02,NaN,0.41,20.73,0.02,NaN,NaN
9,Barangay 6,40,0.04,8.47766,124.64141,0.42,16.65,440.01,NaN,0.44,17.75,468.95,NaN,0.43,17.29,457.04,NaN,0.41,16.25,429.44,NaN,0.41,16.28,430.18,NaN,0.42,16.72,441.75,NaN,NaN


In [30]:
THRESHOLD = 4300  # kg/day/km^2

months = ["jun","jul","aug","sep","oct","nov"]

for m in months:
    df[f"{m}_is_hotspot"] = (df[f"{m}_density"] > THRESHOLD).astype(int)

In [31]:
density_cols = [f"{m}_density" for m in months[:-1]]  # Jun → Oct
df[density_cols] = df[density_cols] * np.random.normal(1, 0.05, df[density_cols].shape)

In [32]:
# Use population, land area, latitude, longitude + last 3 months as features
feature_cols = ["population","land_area_km2","latitude","longitude",
                "aug_density","sep_density","oct_density"]

X_train = df[feature_cols]
y_train = df["nov_is_hotspot"]

In [33]:
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=5,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)
model.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, min_samples_leaf=2, min_samples_split=5,
                       random_state=42)

In [34]:
y_pred = model.predict(X_train)
print("Confusion Matrix:")
print(confusion_matrix(y_train, y_pred))
print("\nClassification Report:")
print(classification_report(y_train, y_pred))

Confusion Matrix:
[[60  0]
 [ 0 20]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        60
           1       1.00      1.00      1.00        20

    accuracy                           1.00        80
   macro avg       1.00      1.00      1.00        80
weighted avg       1.00      1.00      1.00        80



In [35]:
X_dec = df[feature_cols].copy()
X_dec["oct_density"] = df["dec_density"]  # last month replaced by December

df["dec_predicted_hotspot"] = model.predict(X_dec)

# Save December predictions
df[["barangay","dec_density","dec_predicted_hotspot"]].to_csv(
    "data/december_predictions.csv", index=False
)

In [36]:
# Center on CDO
m = folium.Map(location=[8.482, 124.647], zoom_start=12)

for _, row in df.iterrows():
    color = "red" if row["dec_predicted_hotspot"] == 1 else "green"
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=8,
        color=color,
        fill=True,
        fill_opacity=0.7,
        popup=f"{row['barangay']} - {'HOTSPOT' if row['dec_predicted_hotspot'] else 'NORMAL'}"
    ).add_to(m)

m  # display map in Jupyter notebook

In [37]:
import joblib
joblib.dump(model, "rf_hotspot.pkl")


['rf_hotspot.pkl']

In [27]:
m = folium.Map(location=[8.48, 124.65], zoom_start=12)
m.save("default_map.html")